In [1]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-07-07 18:37:23--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: 'documents.json'

     0K .......... .......... .......... .......... ..........  7%  261K 2s
    50K .......... .......... .......... .......... .......... 15%  874K 1s
   100K .......... .......... .......... .......... .......... 23%  525K 1s
   150K .......... .......... .......... .......... .......... 31%  711K 1s
   200K .......... .......... .......... .......... .......... 38%  865K 1s
   250K .......... .......... .......... .......... .......... 46% 1.09M 1s
   300K .......... .......... .......... .......... .......... 54% 8.35M 0s
   350K .........

In [2]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

len(documents)

948

In [3]:
from sentence_transformers import SentenceTransformer

In [7]:
model = SentenceTransformer("all-MiniLM-L12-v2")

c:\Users\USER\.conda\envs\ai\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


From the docs, model size is 120MB while the Avg Performance is 59.76

In [8]:
len(model.encode("This is a simple sentence"))

384